# Lab 02 (Traveling Salesman Problem)

Using Christofides Algorithm to compute a 3/2-approximation

In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from geopy.distance import geodesic
import networkx as nx

## Initialization

In [2]:
# Function to read coordinates from a CSV file
def read_coordinates_from_csv(file_path):
    dtype_dict = {'city': 'string', 'lat': 'float64', 'lon': 'float64'}
    df = pd.read_csv(file_path, header=None, names=['city', 'lat', 'lon'], dtype=dtype_dict, low_memory=False)
    coords = df[['lat', 'lon']].values
    return coords


def create_graph_from_coords(coords):
    """Create a complete graph with distances as weights"""
    G = nx.complete_graph(len(coords))

    for i, (x1, y1) in enumerate(coords):
        for j, (x2, y2) in enumerate(coords):
            if i != j:
                distance = geodesic((x1, y1), (x2, y2)).km
                G[i][j]['weight'] = distance
    return G


def calculate_total_distance(G, path):
    """Calculate total distance of the TSP path"""
    assert path[0] == path[-1]
    assert len(path) - 1 == len(set(path))

    total_distance = 0
    for i in range(len(path) - 1):
        u, v = path[i], path[i + 1]
        total_distance += G[u][v]['weight']
    return total_distance


## Christofides Algorithm

### My Implementation

In [6]:
def minimum_spanning_tree(G):
    """Create a Minimum Spanning Tree"""
    return nx.minimum_spanning_tree(G)

def odd_degree_vertices(G):
    """Find vertices with odd degree"""
    return [v for v in G.nodes() if G.degree(v) % 2 == 1]

def minimum_weight_matching(G, odd_vertices):
    """Find a minimum-weight perfect matching for the odd-degree vertices"""
    subgraph = G.subgraph(odd_vertices)
    min_matching = nx.algorithms.matching.min_weight_matching(subgraph)
    return list(min_matching)

def multigraph_from_mst_and_matching(mst, matching_edges):
    """Combine Minimum Spanning Tree and Perfect Matching"""
    multigraph = nx.MultiGraph(mst)
    multigraph.add_edges_from(matching_edges)
    return multigraph

def eulerian_circuit(G):
    """Find Eulerian Circuit"""
    return list(nx.eulerian_circuit(G))

def tsp_solver_mine(G):
    mst = minimum_spanning_tree(G)
    odd_vertices = odd_degree_vertices(mst)
    matching = minimum_weight_matching(G, odd_vertices)
    multigraph = multigraph_from_mst_and_matching(mst, matching)
    eulerian_circ = eulerian_circuit(multigraph)
    
    # Convert Eulerian circuit to Hamiltonian circuit by skipping duplicates
    visited = set()
    tsp_path = []
    for u, v in eulerian_circ:
        if u not in visited:
            tsp_path.append(u)
            visited.add(u)
    tsp_path.append(tsp_path[0])  # Return to the starting node
    
    return tsp_path

### Official Implementation

In [4]:
def tsp_solver_official(G):
    return nx.algorithms.approximation.christofides(G)

In [7]:
csv_file = 'cities/vanuatu.csv'
coords = read_coordinates_from_csv(csv_file)

G = create_graph_from_coords(coords)

# Use tsp_solver_official to test the internal implementation by networkx
tsp_path = tsp_solver_mine(G)

# Results
total_distance = calculate_total_distance(G, tsp_path)
print("TSP Path:", tsp_path)
print("Total Distance Travelled:", total_distance)

TSP Path: [0, 3, 5, 6, 2, 4, 1, 7, 0]
Total Distance Travelled: 1397.4426516961423
